In [1]:
#High dimensional model 6
#Date: 06/13/17

import numpy as np 

#sigmoid function
def f(x):
  return (1/(1+np.exp(-x)))


#the derivative of f
def df(x):
  return (1/(np.exp(-x)+np.exp(x)+2))


#calculate gradient
def g(Xb,W1,row,a,n):
  Xb = np.repeat(Xb[...,np.newaxis], n, axis=1)
  Xb = Xb-Xb.T
  dXbeta=df(Xb*a)
  xk_xk = np.repeat(row[...,np.newaxis], n, axis=1)
  xk_xk = xk_xk-xk_xk.T
  return (-a*sum(sum(W1*dXbeta*xk_xk))/(n*(n-1)))


#soft-thresholding operator
def Soft_thres(x,lbd_t):
  if x>lbd_t:
    return (x-lbd_t)
  elif x+lbd_t<0:
    return (x+lbd_t)
  else:
    return 0


#grid search for intercept
def h(W, Xbeta, t):
  return sum(W[Xbeta>t])


#grid search for alpha
def alpha_grid(x):
  return 2.0**(x/2+0.5)


#grid search for lambda
def lambda_grid(x):
  return (1+(x-1.0)/3)


#IPW
def ipw(Y1, A1, xbeta1, c1):
  return np.mean(Y1[A1==(xbeta1>c1)])



In [4]:

#coordinate descent algorithm to optimize beta
def solve_beta(X, W, d, n, a, lbd, L, max_iter, converge):
  W1 = np.repeat(W[...,np.newaxis], n, axis=1)
  W1 = W1 - W1.T

  
  beta = np.zeros(d)
  j0 = 0
  beta[j0] = 1


  old_beta = np.copy(beta)
  for j in np.delete(range(d),j0):
      row = X[j,:]
      Xbeta = np.matmul(beta,X)
      grad = g(Xbeta,W1,row,a,n)
      beta[j] = Soft_thres(beta[j]-grad/L, lbd/L)

  i=1
  while (i<max_iter): 

      old_beta = np.copy(beta)
      i = i+1
      for j in np.delete(range(d),j0):
          row = X[j,:]
          Xbeta = np.matmul(beta,X)
          grad = g(Xbeta,W1,row,a,n)
          beta[j] = Soft_thres(beta[j]-grad/L, lbd/L)

      if (np.linalg.norm(beta-old_beta)<converge) and (i>4):
        break
      
  #print(i)
  return(beta)



In [6]:
import pandas as pd
import time
start=time.time()

#numsim is the times of simulations
numsim=100

d=500
n=100
rho=0
#sigma
if rho==0:
    sig=np.identity(d)
else:
    sig=np.array([i for i in range(d)])
    sig=np.repeat(sig[...,np.newaxis], d, axis=1)
    sig=abs(sig-sig.T)
    sig=rho**sig


testn=1000
beta0 = np.zeros(d)
beta0[0]=1
beta0[1]=0.9
beta0[5]=-0.8
beta0=beta0/np.linalg.norm(beta0)
zero_index = [2,3,4]
for i in range(6,d):
  zero_index.append(i)

#the simulation results
simresult=pd.DataFrame(data=np.zeros((numsim,8)), columns=["L2diff^2","Incorr0(0)",
                                    "Corr0(497)", "Within PCD","Estimated Value",
                                    "lambda","alpha","testPCD"])

#cross validation
ncv=int(n/2)
cvindex=np.array([i for i in range(n)]).reshape(ncv,2)


#simulations
for t in range(numsim):
  print("simulation "+str(t))


  #generate samples
  X = np.random.multivariate_normal(np.zeros(d),sig,n).T
  A = np.random.binomial(1,0.5,n)

  Y=1-np.sin(X[0,:])+np.sin(X[1,:])+np.random.normal(0,1,n)+A*(X[0,:]+0.9*X[1,:]-0.8*X[5,:])**3
  v=np.mean(Y[A==0])
  W=4*(Y-v)*(A-0.5)

  #cross validation
  numl=2
  numa=2
  cvresult=np.zeros((numl,numa))
  for l1 in range(numl):
    for a1 in range(numa):
      cvvalue=0
      for t1 in range(2):
        Xcv=X[:,cvindex[:,1-t1]]
        Wcv=W[cvindex[:,1-t1]]
        adjustment = np.sqrt(np.sqrt(ncv/np.log(d)))
        a_cv = adjustment*alpha_grid(a1)
        lbd_cv = lambda_grid(l1)*a_cv*np.sqrt(np.log(d)/ncv)
        beta_cv = solve_beta(Xcv,Wcv,d,ncv,a=a_cv,lbd=lbd_cv,L=10*a_cv,max_iter=1000,converge=np.sqrt(d)*2e-4)

        #calculate cv value
        Xbeta_cv = np.matmul(beta_cv,X[:,cvindex[:,t1]])
        candidate = list(Xbeta_cv)
        candidate.append(min(Xbeta_cv)-0.01)

        c_cv_objective = list(map(lambda x: ipw(Y1=Y[cvindex[:,t1]],
                                           A1=A[cvindex[:,t1]],xbeta1=Xbeta_cv,c1=x), candidate)) 

        cvvalue = cvvalue+max(c_cv_objective)

      cvresult[l1,a1]=cvvalue
    
  
  #print(cvresult)
  max_index = np.argmax(cvresult)
  l1=max_index//numa
  a1=max_index%numa
  #print(l1,a1)

  adjustment = np.sqrt(np.sqrt(n/np.log(d)))
  a=adjustment*alpha_grid(a1)
  lbd=lambda_grid(l1)*a*np.sqrt(np.log(d)/n)

  simresult.iloc[t,5]=lbd
  simresult.iloc[t,6]=a
  beta = solve_beta(X,W,d,n,a=a,lbd=lbd,L=10*a,max_iter=1000,converge=np.sqrt(d)*2e-4)
  #print(beta)

  #calculate L2_diff
  l2_norm = np.linalg.norm(beta)
  if l2_norm==0:
      beta1 = beta
  else:
      beta1 = beta/l2_norm
  
  simresult.iloc[t,0]=np.linalg.norm(beta1-beta0)

  #Incorr0(0)
  simresult.iloc[t,1]=len(list(set(np.where(beta==0)[0])&set([0,1,5])))
  
  #Corr0(48)
  simresult.iloc[t,2]=len(list(set(np.where(beta==0)[0])&set(zero_index)))
  
  #estimate threshold
  Xbeta=np.matmul(beta,X)
  candidate = list(Xbeta)
  candidate.append(min(Xbeta)-0.01)
  c_cv_objective = list(map(lambda x: ipw(Y1=Y,A1=A,xbeta1=Xbeta,c1=x), candidate)) 
  max_index = np.argmax(c_cv_objective)
  b0 = candidate[max_index]
  
  #PCD within these n samples
  sample1=set(np.where(Xbeta>b0)[0])
  truesample1=set(np.where(X[0,:]+0.9*X[1,:]-0.8*X[5,:]>0)[0])
  simresult.iloc[t,3]=(len(sample1&truesample1)+n-len(sample1|truesample1))/n
  
  #Estimated value function
  testX=np.random.multivariate_normal(np.zeros(d),sig,testn).T
  testXbeta=np.matmul(beta,testX)
  testA=np.zeros(testn) 
  testA[testXbeta>b0]=1 
  testY=1-np.sin(testX[0,:])+np.sin(testX[1,:])+testA*(testX[0,:]+0.9*testX[1,:]-0.8*testX[5,:])**3
  simresult.iloc[t,4]=np.mean(testY)
  
  #test PCD
  test1=set(np.where(testXbeta>b0)[0]) 
  true1=set(np.where(testX[0,:]+0.9*testX[1,:]-0.8*testX[5,:]>0)[0])
  simresult.iloc[t,7]=(len(test1&true1)+testn-len(test1|true1))/testn
  

print(simresult[["L2diff^2","Within PCD","lambda","alpha"]])
print(simresult[["Incorr0(0)","Corr0(497)","Estimated Value","testPCD"]])
print(simresult[["L2diff^2","Within PCD","lambda","alpha"]].mean(axis=0))
print(simresult[["Incorr0(0)","Corr0(497)","Estimated Value","testPCD"]].mean(axis=0))

print('sd for estimated value', np.std(simresult.values[:,4]))
print('sd for estimated testPCD', np.std(simresult.values[:,7]))
print('time cost:', time.time()-start)


simulation 0
simulation 1
simulation 2
simulation 3
simulation 4
simulation 5
simulation 6
simulation 7
simulation 8
simulation 9
simulation 10
simulation 11
simulation 12
simulation 13
simulation 14
simulation 15
simulation 16
simulation 17
simulation 18
simulation 19
simulation 20
simulation 21
simulation 22
simulation 23
simulation 24
simulation 25
simulation 26
simulation 27
simulation 28
simulation 29
simulation 30
simulation 31
simulation 32
simulation 33
simulation 34
simulation 35
simulation 36
simulation 37
simulation 38
simulation 39
simulation 40
simulation 41
simulation 42
simulation 43
simulation 44
simulation 45
simulation 46
simulation 47
simulation 48
simulation 49
simulation 50
simulation 51
simulation 52
simulation 53
simulation 54
simulation 55
simulation 56
simulation 57
simulation 58
simulation 59
simulation 60
simulation 61
simulation 62
simulation 63
simulation 64
simulation 65
simulation 66
simulation 67
simulation 68
simulation 69
simulation 70
simulation 71
si